##**`Project: Impact of Credit Score Badge on Loan Default Rate`**

##**Business Scenario**
A company wants to test whether highlighting “Prime Borrower” status for applicants with high credit scores correlates with lower default rates.
The outcome could influence how products are tailored and promoted to high-credit applicants

In [1]:
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest, confint_proportions_2indep


from google.colab import files
uploaded = files.upload()


Saving loan_data.csv to loan_data.csv


In [2]:
# Download dataset
data = pd.read_csv('loan_data.csv')
df = pd.DataFrame(data)
df.head(15)

,Age,Income,Gender,Education,Credit Score,Default
0,58,54504,Male,Masters,784,0.0
1,48,37114,Male,PhD,358,1.0
2,34,108859,Female,Highschool,460,0.0
3,27,129657,Female,Highschool,505,0.0
4,40,40975,Male,Masters,311,1.0
5,58,41023,Male,Highschool,789,0.0
6,38,51447,Female,Bachelors,575,0.0
7,42,120469,Male,PhD,541,0.0
8,30,53959,Male,Bachelors,515,0.0
9,30,96203,Male,Highschool,550,0.0


**Step 1: A/B Group Definition**
* Variant A: Credit Score < median → no badge

* Variant B: Credit Score ≥ median → “Prime Borrower” badge

In [6]:
median_score = df['Credit Score'].median()

df['variant'] = df['Credit Score'].apply(lambda x: 'B' if x >= median_score else 'A')
df.head(15)

,Age,Income,Gender,Education,Credit Score,Default,variant
0,58,54504,Male,Masters,784,0.0,B
1,48,37114,Male,PhD,358,1.0,A
2,34,108859,Female,Highschool,460,0.0,A
3,27,129657,Female,Highschool,505,0.0,A
4,40,40975,Male,Masters,311,1.0,A
5,58,41023,Male,Highschool,789,0.0,B
6,38,51447,Female,Bachelors,575,0.0,A
7,42,120469,Male,PhD,541,0.0,A
8,30,53959,Male,Bachelors,515,0.0,A
9,30,96203,Male,Highschool,550,0.0,A


**Step 2: Default Rate Comparison**

In [7]:
# Calculate default rate comparison

grouped = df.groupby('variant')['Default'].agg(['sum', 'count'])
grouped['rate'] = grouped['sum'] / grouped['count']
print(grouped)

          sum  count      rate
variant                       
A        61.0    299  0.204013
B        47.0    301  0.156146


Purpose of Default Rate Comparison

Measure Risk: Identify which borrower segments are more likely to default

Test Hypotheses: See if certain features (like “Prime Borrower” status) correlate with lower default rates

Optimize Products: Refine loan terms, credit limits, or interest rates based on risk profiles

Improve Marketing: Target safer segments with better offers or messaging

**Step 3: Hypothesis Testing (Two-Proportion Z-Test)**
* H₀: Default rate A = Default rate B (null hypothesis)

* H₁: Default rate B < Default rate A (alternative hypothesis)

In [8]:
successes = [47, 61]
samples = [301, 299]

z_stat, p_val = proportions_ztest(count=successes, nobs=samples, alternative='smaller')
print(f"Z-statistic: {z_stat:.2f}, P-value: {p_val:.4f}")

Z-statistic: -1.53, P-value: 0.0635


**Result:**

Z-statistic: −1.53

P-value: 0.0635

Since p = 0.0635 > 0.05, the result is not statistically significant at the 5% level.

Therefore, **we do not reject the null hypothesis**.

However, because p < 0.10, the result is suggestive — it shows a trend toward significance and may warrant further testing

**Suggestion After Test**

While the difference in default rates between Variant A and B is not statistically significant at the 5% level, the result is suggestive (p = 0.0635) and may indicate a meaningful effect. Further testing with a larger sample size is recommended to validate this trend

In [9]:
# Group B first (lower default rate), then Group A
confint = confint_proportions_2indep(47, 301, 61, 299, method='score', compare='diff')
print(f"95% CI for difference in default rates: {confint}")

95% CI for difference in default rates: (-0.1087929306728726, 0.013760105145952413)


The 95% confidence interval for the difference in default rates between Variant B and Variant A ranges from −10.88% to +1.38%. This means we cannot be 95% confident that Variant B performs better, though the interval suggests a potential reduction in default risk. Further testing with a larger sample is recommended to confirm the effect